In [41]:
import pandas as pd
from datetime import time
import numpy as np

In [42]:
patient_info=pd.read_csv('AsthmaFiles/patient_info.csv')
smartwatch1=pd.read_csv('AsthmaFiles/smartwatch1.csv')
smartwatch2=pd.read_csv('AsthmaFiles/smartwatch2.csv')
smartwatch3=pd.read_csv('AsthmaFiles/smartwatch3.csv')

In [43]:
patient_info[["user_key"]+list(patient_info)[-11:-2]]

,user_key,miband_start_date,position_start_date,pef_start_date,inhaler_start_date,daily_end_date,weekly_end_date,miband_end_date,position_end_date,pef_end_date
0,113,0.0,1,1.0,0.0,183,183,183.0,183,40.0
1,190,1.0,1,6.0,1.0,183,182,148.0,183,183.0
2,217,0.0,0,0.0,1.0,1,0,1.0,1,0.0
3,278,10.0,0,NaN,-20.0,15,15,15.0,13,NaN
4,294,-1.0,0,0.0,-5.0,183,182,183.0,183,183.0
5,328,6.0,0,0.0,0.0,30,14,12.0,30,30.0
6,343,0.0,0,1.0,0.0,183,182,183.0,183,183.0
7,398,0.0,0,NaN,0.0,36,36,36.0,36,NaN
8,447,0.0,0,0.0,-1.0,126,126,91.0,126,90.0
9,454,0.0,0,NaN,0.0,10,8,8.0,10,NaN


In [44]:
list_of_user_keys=[]
for x in patient_info[["user_key"]+list(patient_info)[-10:-2]].iterrows():
    if x[1]["pef_end_date"] - x[1]["pef_start_date"] >= 50 and x[1]["miband_end_date"]!="NaN":
        list_of_user_keys.append(x[1]["user_key"])
list_of_user_keys

[190.0, 294.0, 343.0, 447.0, 473.0, 514.0, 625.0, 702.0, 808.0, 939.0]

In [45]:
def get_peakflow_data(user_key):
    peakflow_data = pd.read_csv("AsthmaFiles/peakflow.csv")
    peakflow_data = peakflow_data[peakflow_data["user_key"]==user_key]
    peakflow_data = peakflow_data[["date","hour","pef_max"]]
    return peakflow_data


In [46]:
def pair_weather(id,dates):
    weather = pd.read_csv("AsthmaFiles/environment.csv")
    for_id = weather.loc[weather['user_key'] == id]
    weather = for_id.loc[for_id['date'].isin(dates)]

    return weather

In [47]:
peak_flow_data = get_peakflow_data(list_of_user_keys[0])
weather = pair_weather(list_of_user_keys[0],peak_flow_data["date"])


In [48]:
def try_catch(row,default,peak_flow_data,weather,x):
    try:
        return weather.loc[weather['date'] == row["date"]].iloc[0][x]
    except IndexError:
        return default

In [49]:

# seperate_for_key = smartwatch1.loc[smartwatch1["user_key"]==list_of_user_keys[0]][["date","time","hr"]]
# seperate_for_date = seperate_for_key.loc[seperate_for_key["date"]==1][["time","hr"]]
# seperate_for_time = seperate_for_date.loc[seperate_for_date["time"].str.startswith("08")]["hr"]

def seperate_for_key(user_key,smartwatch1,smartwatch2,smartwatch3):
    seperate_for_key1 = smartwatch1.loc[smartwatch1["user_key"]==user_key][["date","time","hr"]]
    seperate_for_key2 = smartwatch2.loc[smartwatch2["user_key"]==user_key][["date","time","hr"]]
    seperate_for_key3 = smartwatch3.loc[smartwatch3["user_key"]==user_key][["date","time","hr"]]
    frames = [seperate_for_key1, seperate_for_key2, seperate_for_key3]
    return pd.concat(frames)

def seperate_for_date(date,seperate_for_key):
    return seperate_for_key.loc[seperate_for_key["date"]==date][["time","hr"]]

def seperate_for_time(time,seperate_for_date):
    if len(str(time)) == 1:
        time = "0" + str(time)
    return seperate_for_date.loc[seperate_for_date["time"].str.startswith(str(time))]["hr"].max()


def confusing_method(row,key,sm1,sm2,sm3):
    sk = seperate_for_key(key,sm1,sm2,sm3)
    sd = seperate_for_date(row["date"],sk)
    return seperate_for_time(row["hour"],sd)



In [50]:
default = "NA"
for i in list_of_user_keys:
    peak_flow_data = get_peakflow_data(i)
    weather = pair_weather(i,peak_flow_data["date"])
    for x in list(weather.columns):
        if x not in peak_flow_data.columns:
            peak_flow_data[x] = peak_flow_data.apply(lambda row: try_catch(row,default,peak_flow_data,weather,x), axis = 1)
    peak_flow_data["hr"] = peak_flow_data.apply(lambda row: confusing_method(row,i,smartwatch1,smartwatch2,smartwatch3), axis = 1)
    peak_flow_data.to_csv(f"AsthmaFiles/{int(i)}.csv",index=False)

In [80]:

import pandas as pd

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

def get_csv_for_id(id):
    dataset = pd.read_csv(f"AsthmaFiles/{id}.csv")
    z = dataset["pef_max"].max()
    dataset = dataset.drop(["weed_pollen","tree_pollen","grass_pollen"],axis=1)
    dataset = clean_dataset(dataset)
    y = dataset["pef_max"].apply(lambda x: x/z)
    x = dataset.drop(["pef_max"],axis=1)
    return x,y

x,y=get_csv_for_id(190)


0      0.689441
1      0.608696
2      0.556936
3      0.565217
4      0.681159
         ...   
199    0.782609
200    0.732919
201    0.712215
202    0.807453
206    0.689441
Name: pef_max, Length: 184, dtype: float64

In [68]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=100)


In [69]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)





LinearRegression()

In [70]:
y_lr_test_pred = lr.predict(x_train)
y_lr_test_pred

array([0.70038663, 0.68820851, 0.65886089, 0.67044316, 0.6043551 ,
       0.67124286, 0.68604048, 0.6464544 , 0.71683754, 0.69072815,
       0.69814772, 0.74887341, 0.68675139, 0.64932444, 0.65807214,
       0.70809025, 0.67019962, 0.69028925, 0.69114941, 0.64352072,
       0.60161265, 0.65577033, 0.59009362, 0.64108971, 0.65849008,
       0.64050203, 0.67538419, 0.68001025, 0.71742416, 0.65323041,
       0.61527054, 0.65540664, 0.69603235, 0.66801151, 0.69563993,
       0.69737727, 0.59278146, 0.70993779, 0.65813737, 0.68574163,
       0.66530941, 0.66440386, 0.66717276, 0.62218731, 0.67913424,
       0.66099426, 0.61834607, 0.64435401, 0.6601482 , 0.67947804,
       0.63147199, 0.69632371, 0.70503518, 0.63236123, 0.69829155,
       0.68013861, 0.62515188, 0.63493287, 0.61773736, 0.68913493,
       0.65380843, 0.62846994, 0.69630041, 0.63787516, 0.63749679,
       0.65156019, 0.67547822, 0.64254231, 0.64299423, 0.70827653,
       0.65501647, 0.6892543 , 0.68668201, 0.67705507, 0.66486

In [71]:
y_lr_train_pred = lr.predict(x_test)
y_lr_train_pred

array([0.66885799, 0.66430262, 0.62827948, 0.67579813, 0.65901327,
       0.60416785, 0.67739639, 0.67194413, 0.6787591 , 0.66061749,
       0.69245992, 0.68326676, 0.70753459, 0.68264953, 0.68301234,
       0.65410298, 0.62417406, 0.65177473, 0.61280238, 0.67621814,
       0.64234078, 0.66103641, 0.7009346 , 0.70183852, 0.67081785,
       0.65263701, 0.63931944, 0.68159582, 0.67505031, 0.69842176,
       0.68648415, 0.68871354, 0.73163839, 0.66894199, 0.64438297,
       0.70084857, 0.69866914, 0.66315639, 0.71789819, 0.6440804 ,
       0.62142544, 0.63595416, 0.68279477, 0.7342208 , 0.6581273 ,
       0.65138757, 0.71657232, 0.63263514, 0.65253696, 0.64000886,
       0.66362016, 0.64805784, 0.63277826, 0.67070632, 0.6660385 ,
       0.68039202])

In [77]:
from sklearn.metrics import mean_squared_error, r2_score

lr_train_mse = mean_squared_error(y_train, y_lr_test_pred)
lr_train_r2 = r2_score(y_train, y_lr_test_pred)

lr_test_mse = mean_squared_error(y_test, y_lr_train_pred)
lr_test_r2 = r2_score(y_test, y_lr_train_pred)

lr_train_mse, lr_train_r2, lr_test_mse, lr_test_r2



(0.0031233375423679734,
 0.2302966646560256,
 0.005337856810992749,
 -0.014192882810396545)